## I.2 Explore and perform Data Wrangling on:

1. Dataset: Scrapped Dataset of Properties listed in Birmingham B28 + 1 mile Radius (rightmove.co.uk)
2. Scrapped date: 4, Nov, 2024
3. Author: Lam H. Tong

In [1]:
# Import packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import geopy

In [2]:
# Import scrapped dataset

# All properties listed
df_all     = pd.read_csv('[all]B28_plus_1mile_rightmove_4_Nov.csv')

# Properties with garden only
df_garden = pd.read_csv('[garden]B28_plus_1mile_rightmove_4_Nov.csv')

# Properties with parking only
df_parking = pd.read_csv('[parking]B28_plus_1mile_rightmove_4_Nov.csv')

# New home only
df_new    = pd.read_csv('[new]B28_plus_1mile_rightmove_4_Nov.csv')

In [3]:
# The main dataframe info before wrangling
df_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 752 entries, 0 to 751
Data columns (total 7 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   address  752 non-null    object
 1   price    752 non-null    object
 2   type     752 non-null    object
 3   no_bed   752 non-null    object
 4   no_bath  752 non-null    object
 5   agent    752 non-null    object
 6   url      752 non-null    object
dtypes: object(7)
memory usage: 41.3+ KB


In [4]:
# remove excessive characters
for df in [df_garden, df_new, df_parking, df_all]:
    df['price'] = df['price'].str.replace('£', '')

# drop duplicates (as every scrapped pages have 1 same advertised property):
df_all = df_all.drop_duplicates()
df_garden = df_garden.drop_duplicates()
df_new = df_new.drop_duplicates()
df_parking = df_parking.drop_duplicates()

In [5]:
# Adding featured coulumns in the filtered dataset 
df_parking['parking'] = True
df_garden['garden'] = True
df_new['new_home'] = True

In [6]:
#change 'no_bed' type to string to perform merging bc sh only this feild of this particular dataset is int64
df_garden['no_bed'] = df_garden['no_bed'].astype(str)

# merge the df_all with df_garden first
df_temp1 = pd.merge(df_all, df_garden, on = ['address', 'price', 'type', 'no_bed', 'no_bath', 'agent', 'url'] , how = 'outer')

# merge with df_parking
df_temp2 = pd.merge(df_temp1, df_parking, on = ['address', 'price', 'type', 'no_bed', 'no_bath', 'agent', 'url'] , how = 'outer')

#change 'no_bed' type to string to perform merging
df_new['no_bed'] = df_new['no_bed'].astype(str)

# the final merged dataframe
df = pd.merge(df_temp2, df_new, on = ['address', 'price', 'type', 'no_bed', 'no_bath', 'agent', 'url'] , how = 'outer')

In [7]:
# Define the regex pattern for UK postcodes
postcode_pattern = r'(\bB\d{1,2}(?:\s?\d[A-Z]{2})?\b)'

# Extract postcodes using .str.extract() and the pattern if there're any
df['postcode'] = df['address'].str.extract(postcode_pattern)

# Replace excessive newlines and carriage returns with a single space
df['address'] = df['address'].str.replace(r'[\n\r]+', ' ', regex=True)

In [8]:
# Replace 'not_specified' with NaN for `no_bed` and `no_bath`
df['no_bed'].replace('not_specified', np.nan, inplace=True)
df['no_bath'].replace('not_specified', np.nan, inplace=True)

# Remove commas and convert `price` to int
df['price'] = pd.to_numeric(df['price'].str.replace(',', ''), errors='coerce')

# Convert `no_bed` and `no_bath` to numeric, allowing NaNs
df['no_bed'] = pd.to_numeric(df['no_bed'], errors='coerce')
df['no_bath'] = pd.to_numeric(df['no_bath'], errors='coerce')

# Convert to nullable integer type (Int64) to handle NaNs
df['no_bed'] = df['no_bed'].astype('Int64')  # Nullable int
df['no_bath'] = df['no_bath'].astype('Int64')  # Nullable int

# Replacing Nulls in 'garden', 'parking', and 'new_home' as False
for feature in ['garden', 'parking', 'new_home']:
    df.loc[df[feature].isna(), feature] = False

C:\Users\blueb\AppData\Local\Temp\ipykernel_19744\2775976841.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['no_bed'].replace('not_specified', np.nan, inplace=True)
C:\Users\blueb\AppData\Local\Temp\ipykernel_19744\2775976841.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a cop

In [13]:
df[df['no_bath'].isna()]

,address,price,type,no_bed,no_bath,agent,url,garden,parking,new_home,postcode
0,"121 Brook Lane, Birmingham B13 0AB",99500,Terraced,2,<NA>,"SDL Property Auctions, Nationwide",https://www.rightmove.co.uk/properties/154574144#/?channel=RES_BUY,False,True,False,B13 0AB
71,"Bradnock Close, Birmingham, West Midlands, B13",250000,Semi-Detached,3,<NA>,"Dixons, Kings Heath",https://www.rightmove.co.uk/properties/154066088#/?channel=RES_BUY,True,True,False,B13
79,"Broadoaks, Streetsbrook Road, Solihull",180000,Studio,1,<NA>,"Set To Let, Leicester",https://www.rightmove.co.uk/properties/145873511#/?channel=RES_BUY,False,True,False,NaN
82,"Bromwall Road, Birmingham, B13",245000,End of Terrace,3,<NA>,"Express Estate Agency, Nationwide",https://www.rightmove.co.uk/properties/151981685#/?channel=RES_BUY,True,True,False,B13
219,"Garages to the rear of, 9-49 Ravenshill Road, Birmingham, B14 4HL",225000,Land for sale,<NA>,<NA>,"Bond Wolfe, Commercial Sales",https://www.rightmove.co.uk/properties/150797960#/?channel=COM_BUY,False,True,False,B14 4HL
220,"Garden Bungalow, 31 Oxford Road, Moseley, Birmingham, B13 9EH",140000,Bungalow,2,<NA>,"Bond Wolfe, Auctions",https://www.rightmove.co.uk/properties/153279509#/?channel=RES_BUY,True,True,False,B13 9EH
303,"James Court, Wake Green Park",100000,Flat,1,<NA>,"Rice Chamberlains LLP, Moseley",https://www.rightmove.co.uk/properties/153012953#/?channel=RES_BUY,False,True,False,NaN
433,"Princethorpe Close, Shirley, Solihull, B90",450000,Semi-Detached,6,<NA>,"Urban Edge Estates, Covering Shirley",https://www.rightmove.co.uk/properties/86928021#/?channel=RES_BUY,True,True,False,B90
494,"Shaftmoor Lane, Hall Green, Birmingham, West Midlands, B28 8SW",320000,End of Terrace,2,<NA>,David Wilson Homes,https://www.rightmove.co.uk/properties/154177844#/?channel=RES_BUY,True,True,True,B28 8SW
495,"Shaftmoor Lane, Hall Green, Birmingham, West Midlands, B28 8SW",399995,Semi-Detached,3,<NA>,David Wilson Homes,https://www.rightmove.co.uk/properties/149527079#/?channel=RES_BUY,True,True,True,B28 8SW


In [14]:
# Correcting Bathrooms manunally as per the provided floor plan AND description of each property
for idx in [0, 79, 82, 220, 303, 639, 640, 655]:
    df.loc[idx, 'no_bath'] = 1

for idx in [71]:
    df.loc[idx, 'no_bath'] = 2

for idx in [433, 494, 495, 496, 497, 498, 499, 500, 501, 502, 503, 504, 505, 506, 507, 508, 509, 510, 548]:
    df.loc[idx, 'no_bath'] = 3

In [18]:
df[df['no_bed'].isna()]

,address,price,type,no_bed,no_bath,agent,url,garden,parking,new_home,postcode


In [17]:
#Remove the above properties as they are commercial buildings
df = df.drop(index = [219, 628])

In [22]:
# Correcting types of these properties
df.loc[58, 'type'] = 'Flat'
df.loc[88, 'type'] = 'Semi-Detached'

# Drop the Commercial Property 
df = df.drop(686, axis = 0)

In [24]:
# Export to a separted dadaset
df.to_csv('B28_properties_dataset_cleaned.csv', index=False)